
# MCP Demo

In [1]:
# Install mcp, langgraph, and langchain if needed (uncomment when running locally)
# !pip install langchain-mcp-adapters langgraph>=0.2.0 langchain-openai mcp

from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

import os
## SET YOUR OPENAI_API_KEY HERE

C:\Users\marci\anaconda3\envs\DATAENLIGHT_AI_LAB\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage


# Helper function to parse and display messages and tool calls
def parse_output(result_dict):
    """
    Parse the output from demo_multi_server() and display the conversation flow
    including user input, tool calls, tool responses, and final agent response.

    Args:
        result_dict: The result dictionary containing 'messages' list
    """
    messages = result_dict.get('messages', [])

    print("=== CONVERSATION FLOW ===")
    print()

    for i, message in enumerate(messages):
        print(f"Step {i + 1}:")

        if isinstance(message, HumanMessage):
            print(f"  USER INPUT: {message.content}")

        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                print(f"  AGENT TOOL CALLS:")
                for tool_call in message.tool_calls:
                    print(f"    - Tool: {tool_call['name']}")
                    print(f"      Arguments: {tool_call['args']}")
                    print(f"      Call ID: {tool_call['id']}")
            else:
                print(f"  AGENT RESPONSE: {message.content}")

        elif isinstance(message, ToolMessage):
            print(f"  TOOL OUTPUT:")
            print(f"    - Tool: {message.name}")
            print(f"    - Result: {message.content}")
            print(f"    - Call ID: {message.tool_call_id}")

        print()


In [3]:

async def demo_single_server():
    """Demonstrate connecting to a filesystem MCP server using MultiServerMCPClient.

    This example launches the official filesystem server from the Model Context Protocol
    project using `npx mcp-server-filesystem`. Update the root path as needed.

    """
    model = ChatOpenAI(model='gpt-4', temperature=0)
    # Define connection configuration for the filesystem server
    connections = {
        "filesystem": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "./ExampleFolder",  # Specify directories that the agent is allowed to access
                # Add more directories as needed
            ],
            "transport": "stdio",
        }
    }
    client = MultiServerMCPClient(connections)

    tools = await client.get_tools(server_name='filesystem')
    print('Available tools:', [t.name for t in tools], '\n\n')

    agent = create_react_agent(model, tools)

    # Example query: list files in the root directory
    result = await agent.ainvoke({
        "messages": [
            {"role": "user", "content": "List all files in ExampleFolder directory"}
        ]
    })


    parse_output(result)



# await demo_single_server()

In [6]:
#await demo_single_server()

In [11]:

async def demo_multi_server():
    """Demonstrate connecting to multiple MCP servers using MultiServerMCPClient.

    This example combines the filesystem server (run via npx) and a local math server
    implemented as a Python script. Update the command paths and root directory
    to match your environment.

    """
    model = ChatOpenAI(model='gpt-4', temperature=0)
    # Define connection configurations for two servers
    connections = {
"""
        "filesystem": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "./ExampleFolder",
                # Add more directories as needed
            ],
            "transport": "stdio",
        },
"""
        'math': {
            'command': 'sys.executable',
            'args': ['./math_server.py'],  # Update with your math server path
            'transport': 'stdio',
        },
    }
    client = MultiServerMCPClient(connections)
    tools = await client.get_tools()  # load tools from both servers
    print('Total tools loaded:', len(tools))
    agent = create_react_agent(model, tools)

    # Example: ask a math question
    math_result = await agent.ainvoke({
        "messages": [
            {"role": "user", "content": "what's (3 + 5) x 12?"}
        ]
    })

    parse_output(math_result)

# await demo_multi_server()

In [12]:
await demo_multi_server()

UnsupportedOperation: fileno